# Datasets:

## Financial Statements

The financial dataset was obtained from SEC Edgar financial statement data set, which includes the company balance sheet, income statement and statement of cash flows. The data is provided quarterly since January 2009 to June 2024, which is the most recent dataset as of the writing of this proposal. SEC (January 2009 - June 2024). The SEC provides this data set using eXtensible Business Reporting Language (XBRL) which divides the dataset amongst many disjoint tables SEC (2024). In order to provide the Large Language model with a single set of tables we will use the following helper tool to process the dataset into a single data frame HansjoergW (2024).From this statement we will then use the following formulas to calculate a comprehensive set of financial ratios that will be provided. From this we will be able to create a dataset similar to that used in Kim et al. (2024).

Github Repo: https://github.com/HansjoergW/sec-fincancial-statement-data-set/tree/main

### Initial Setup

In [ ]:
# to ensure that the logging statements are shown in juypter output, run this cell
import logging
import pandas as pd
from secfsdstools.update import update

logger = logging.getLogger()
logger.setLevel(logging.INFO)

# ensure that all columns are shown and that colum content is not cut
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width',1000)

#Ensure database is up to date with SEC releases
# If have not run for the first time, will take a few minutes to download dataset.
update()


For our first milestone we have focused our efforts in obtaining information from the following Companies: AAPLE, JPMORGAN, EXXON, RATHEON and JOHNSSON & JOHNSSON. In order to query the database, we need to search using the central index key. We will use the following object to store all relevant aspects of the company.

In [ ]:
from secfsdstools.c_index.companyindexreading import CompanyIndexReader

class Company:
    def __init__(self, cik):
        self.cik = cik
        self.report_reader = CompanyIndexReader.get_company_index_reader(cik=self.cik)
    
    def get_cik(self):
        return self.cik

    def get_report_reader(self):
        return self.report_reader
    
    def getAvailableReports(self):
        return list(self.report_reader.get_all_company_reports_df()['form'].unique()) 

    def getFilingList(self, reportType, startDate, endDate):
        if reportType == 'All':
            unfilteredDF = self.report_reader.get_all_company_reports_df()
        else:
            unfilteredDF = self.report_reader.get_all_company_reports_df(forms=reportType)
            
        filteredDF = unfilteredDF[(unfilteredDF.period >= startDate) & (unfilteredDF.period <= endDate)]
        return filteredDF
    

In [ ]:
from secfsdstools.c_index.searching import IndexSearch

companyNames = [
    "Apple Inc",
    "Johnson & Johnson",
    "JPMorgan Chase",
    "Exxon",
    "Lockheed Martin",
    "NVIDIA CORP"
]

companyObjDict = dict()
index_search = IndexSearch.get_index_search()
for c in companyNames:
    results = index_search.find_company_by_name(c)
    companyObjDict[c] = Company(cik=results.iloc[0]['cik'])


Get 10Ks between time periods

In [ ]:
filings_10k_10Q = companyObjDict["NVIDIA CORP"].getFilingList(reportType=["10-K","10-Q"], startDate=0, endDate=20241231)

filings_10k_10Q

# Filtering strategy 

Stategy, use ddate == filing date, remove items with qrts != 1, used first indexed value.


In [58]:
from secfsdstools.e_collector.multireportcollecting import MultiReportCollector
nvdia_Q3_2023_adsh = "0001045810-23-000227"
periodFiled = filings_10k_10Q[filings_10k_10Q.adsh == nvdia_Q3_2023_adsh]



# load only the assets tags that are present in the 10-K report of apple in the years
# 2022 and 2012
#tag_filter = IS, BS, CF

collector: MultiReportCollector = MultiReportCollector.get_reports_by_adshs(
                                              adshs=[nvdia_Q3_2023_adsh], stmt_filter='IS')
rawdatabag = collector.collect()

# as expected, there are just two entries in the submission dataframe
# print(rawdatabag.sub_df, '\n')
dataFrame = rawdatabag.num_df


2024-10-25 18:37:37,391 [INFO] configmgt  reading configuration from /Users/joseluistejada/.secfsdstools.cfg
2024-10-25 18:37:37,395 [INFO] parallelexecution      items to process: 1
2024-10-25 18:37:37,520 [INFO] parallelexecution      commited chunk: 0


,adsh,tag,version,coreg,ddate,qtrs,uom,value,footnote
0,0001045810-23-000227,AccountsPayableCurrent,us-gaap/2023,,20230131,0,USD,1.193000e+09,None
1,0001045810-23-000227,AccountsPayableCurrent,us-gaap/2023,,20231031,0,USD,2.380000e+09,None
2,0001045810-23-000227,AccruedLiabilitiesCurrent,us-gaap/2023,,20230131,0,USD,4.120000e+09,None
3,0001045810-23-000227,AccruedLiabilitiesCurrent,us-gaap/2023,,20231031,0,USD,5.472000e+09,None
4,0001045810-23-000227,AccumulatedOtherComprehensiveIncomeLossNetOfTax,us-gaap/2023,,20230131,0,USD,-4.300000e+07,None
...,...,...,...,...,...,...,...,...,...
439,0001045810-23-000227,PurchaseObligationAndOtherCommitments,0001045810-23-000227,,20231031,0,USD,4.430000e+09,None
440,0001045810-23-000227,PurchaseObligationInventoryPurchaseAndSupplyAndCapacityCommitmentRemainingMinimumAmountsCommitted,0001045810-23-000227,,20231031,0,USD,1.711000e+10,None
441,0001045810-23-000227,PurchaseObligationToBePaidAfterYearFour,0001045810-23-000227,,20231031,0,USD,3.540000e+08,None
442,0001045810-23-000227,StockRepurchaseProgramAdditionalNumberOfSharesAuthorizedToBeRepurchased,0001045810-23-000227,,20230831,0,USD,2.500000e+10,None


In [67]:
print(list(dataFrame.tag))
dataFrame.ddate.value_counts()
dataFrame.ddate.dtype

['AccountsPayableCurrent', 'AccountsPayableCurrent', 'AccruedLiabilitiesCurrent', 'AccruedLiabilitiesCurrent', 'AccumulatedOtherComprehensiveIncomeLossNetOfTax', 'AccumulatedOtherComprehensiveIncomeLossNetOfTax', 'AdditionalPaidInCapital', 'AdditionalPaidInCapital', 'AdjustmentsToAdditionalPaidInCapitalSharebasedCompensationRequisiteServicePeriodRecognitionValue', 'AdjustmentsToAdditionalPaidInCapitalSharebasedCompensationRequisiteServicePeriodRecognitionValue', 'AdjustmentsToAdditionalPaidInCapitalSharebasedCompensationRequisiteServicePeriodRecognitionValue', 'AdjustmentsToAdditionalPaidInCapitalSharebasedCompensationRequisiteServicePeriodRecognitionValue', 'AllocatedShareBasedCompensationExpense', 'AllocatedShareBasedCompensationExpense', 'AllocatedShareBasedCompensationExpense', 'AllocatedShareBasedCompensationExpense', 'AntidilutiveSecuritiesExcludedFromComputationOfEarningsPerShareAmount', 'AntidilutiveSecuritiesExcludedFromComputationOfEarningsPerShareAmount', 'AntidilutiveSecuri

dtype('int64')

In [74]:
import numpy as np
filingPeriod = np.asarray((str(periodFiled["period"].values)[1:-1]), dtype='int64')
filingPeriod

array(20231031)

In [75]:
dataFrame[(dataFrame.ddate == filingPeriod) & (dataFrame.qtrs.isin([0, 1]))]
#Saved

,adsh,tag,version,coreg,ddate,qtrs,uom,value,footnote
1,0001045810-23-000227,AccountsPayableCurrent,us-gaap/2023,,20231031,0,USD,2.380000e+09,None
3,0001045810-23-000227,AccruedLiabilitiesCurrent,us-gaap/2023,,20231031,0,USD,5.472000e+09,None
5,0001045810-23-000227,AccumulatedOtherComprehensiveIncomeLossNetOfTax,us-gaap/2023,,20231031,0,USD,-8.800000e+07,None
7,0001045810-23-000227,AdditionalPaidInCapital,us-gaap/2023,,20231031,0,USD,1.299100e+10,None
10,0001045810-23-000227,AdjustmentsToAdditionalPaidInCapitalSharebasedCompensationRequisiteServicePeriodRecognitionValue,us-gaap/2023,,20231031,1,USD,9.830000e+08,None
...,...,...,...,...,...,...,...,...,...
438,0001045810-23-000227,ProductWarrantyAccrualsAndReturnProvisionsCurrent,0001045810-23-000227,,20231031,0,USD,2.990000e+08,None
439,0001045810-23-000227,PurchaseObligationAndOtherCommitments,0001045810-23-000227,,20231031,0,USD,4.430000e+09,None
440,0001045810-23-000227,PurchaseObligationInventoryPurchaseAndSupplyAndCapacityCommitmentRemainingMinimumAmountsCommitted,0001045810-23-000227,,20231031,0,USD,1.711000e+10,None
441,0001045810-23-000227,PurchaseObligationToBePaidAfterYearFour,0001045810-23-000227,,20231031,0,USD,3.540000e+08,None


# Advanced Statement Standarization


In [ ]:
from secfsdstools.e_collector.reportcollecting import SingleReportCollector
from secfsdstools.e_filter.rawfiltering import ReportPeriodRawFilter, StmtRawFilter
from secfsdstools.e_presenter.presenting import StandardStatementPresenter
from secfsdstools.u_usecases.bulk_loading import default_postloadfilter
from secfsdstools.e_filter.joinedfiltering import StmtJoinedFilter
from secfsdstools.f_standardize.bs_standardize import BalanceSheetStandardizer
from secfsdstools.f_standardize.is_standardize import IncomeStatementStandardizer
from secfsdstools.f_standardize.cf_standardize import CashFlowStandardizer

bs_standardizer = BalanceSheetStandardizer()
is_standardizer = IncomeStatementStandardizer()
cf_standardizer = CashFlowStandardizer()

# initialize the search class
search = IndexSearch.get_index_search()

# create a list with all known forms
forms_list = ['10-12B', '10-12G', '10-12G/A', '10-D', '10-K', '10-K/A', '10-KT', '10-KT/A', '10-Q', '10-Q/A', '10-QT', '10-QT/A', '18-K', '20-F', '20-F/A', '20FR12B', '20FR12G', '40-F', '40-F/A', '424B1', '424B2', '424B3', '424B4', '424B5', '424B7', '425', '6-K', '6-K/A', '8-K', '8-K/A', '8-K12B', '8-K12B/A', '8-K12G3', 'ARS', 'DEF 14A', 'DEF 14C', 'DEFA14A', 'DEFC14A', 'DEFM14A', 'DEFM14C', 'DEFR14A', 'F-1', 'F-1/A', 'F-3', 'F-3/A', 'F-3ASR', 'F-4', 'F-4/A', 'N-2', 'N-2/A', 'N-2ASR', 'N-2MEF', 'N-4', 'N-4/A', 'N-6/A', 'N-CSR', 'N-CSR/A', 'N-CSRS', 'N-CSRS/A', 'NT 10-Q', 'POS 8C', 'POS AM', 'POS AMI', 'POS EX', 'POSASR', 'PRE 14A', 'PREC14A', 'PREM14A', 'PRER14A', 'PRER14C', 'S-1', 'S-1/A', 'S-11', 'S-11/A', 'S-1MEF', 'S-3', 'S-3/A', 'S-3ASR', 'S-4', 'S-4/A', 'SP 15D2']
stmt_list = ['BS', 'CF', 'CI', 'CP', 'EQ', 'IS', 'SI', 'UN']


In [ ]:
adsh = "0001045810-23-000227"
reader = SingleReportCollector.get_report_by_adsh(adsh=adsh, stmt_filter=['BS', 'IS', 'CF'])
raw_data = reader.collect()
raw_data

In [ ]:
filterd_data = raw_data.filter(ReportPeriodRawFilter())
raw_stmts_data = filterd_data.filter(StmtRawFilter(stmts=stmt_list))
joined_df = filterd_data.join()
report_data = joined_df.present(StandardStatementPresenter(invert_negating=True))
 # loading stardized view of BS, IS
std_joined_df = default_postloadfilter(raw_stmts_data).join()

#Standarized Balance Sheet
bs_joined_df = std_joined_df[StmtJoinedFilter(stmts=['BS'])]
bs_standardized = bs_joined_df.present(bs_standardizer)
cols = [x for x in bs_standardized.columns.tolist() if not x.endswith('error')]
bs_standardized[cols]


In [ ]:
#Standarized Income Statement
is_joined_df = std_joined_df[StmtJoinedFilter(stmts=['IS'])]
is_standardized = is_joined_df.present(is_standardizer)
cols = [x for x in is_standardized.columns.tolist() if not x.endswith('error')]
is_standardized[cols]

In [ ]:
cols

In [ ]:
#Standarized Cash Flows
cf_joined_df = std_joined_df[StmtJoinedFilter(stmts=['CF'])]
cf_standardized = cf_joined_df.present(cf_standardizer)
cols = [x for x in cf_standardized.columns.tolist() if not x.endswith('error')]
cf_standardized[cols]